## Retrieving and cleaning data


This notebook can be used to get the data from your experiment saved as a CSV
file. You need to put the SQL file you receive in the phpmyadmin tool (see
Canvas on instructions how to do this) and then run this code. Watch out!
Sometimes you need to make small adaptations (indicated in the notebook)


In [1]:
!pip install sqlalchemy

In [2]:
import pandas as pd
from sqlalchemy import create_engine, inspect

The following code connects to the SQL database from phpmyadmin and gets the
names of all the tables that are present. For each group there should be several
tables such as "calibration", "userconsent" or "propositions" -- but there might
be some columns only present for some of the groups.

**!! IMPORTANT !! You need to change the first command to reflect the name that
you have given the database. In this example the database was called "group25",
replace this with the name the database has in your phpmyadmin**


In [3]:
# Create a connection to the MySQL database using SQLAlchemy
engine = create_engine('mysql+pymysql://root:@localhost/group21_final') # !!! Change database name here !!!

# Use the inspect module to get table names
inspector = inspect(engine)
tables = inspector.get_table_names()

# Print the list of all tables
for table in tables:
    print(table)

group21_calibration
group21_pre_propositions
group21_propositions
group21_propositions1
group21_propositions2
group21_propositions3
group21_propositions4
group21_recommendation
group21_userallergies
group21_userconsent
group21_userdemograph
group21_usermancheck
group21_userpresatisfaction
group21_users
group21_usersatisfaction
group21_usersatisfaction1
group21_usersatisfaction2
group21_usersatisfaction3
group21_usersatisfaction4
group21_userselection


Since we also want to know to which statements the different likert scale items
correspond (important for later analysis), we can retrieve a kind of "codebook"
for this from the database. The information about the likert scale items can be
stored in a few different tables: pre_propositions (asked before the
calibration/recommendation), propositions (asked after the recommendation), and
sometimes propositions1 if there were more questions than fit on the pages. Your
group might have one, two, or all of these tables. Running the code below
outputs the saved (pre-)propositions with their id, question wording, and scale
(usually 5 or 7 point). You can use the id to look up the matching column in the
csv dataframe later.

With the code below we are retrieving the tables, loading them into pandas
dataframes and adding a prefix to the item names (pre* or post*) as often the
IDs are the same for prepropositions and propositions, so we want to be able to
keep them apart later.


In [4]:
def load_and_concat_propositions(engine, group_prefix):
    tables = inspector.get_table_names()  # Retrieve all table names from the database
    # Filter tables to only include those containing "pre_propositions" or "propositions" and match the group prefix
    proposition_names = [table.replace(group_prefix, '') for table in tables if ("pre_propositions" in table or "propositions" in table) and table.startswith(group_prefix)]
    
    dfs = []  # List to store individual DataFrames
    for proposition_name in proposition_names:
        # Determine prefix based on proposition_name pattern
        if "pre_propositions" in proposition_name:
            prefix = 'pre_'
        elif proposition_name.startswith('propositions'):
            prefix = 'post_'
        else:
            prefix = ''  # Default prefix if none of the conditions match

        table_name = f"{group_prefix}{proposition_name}" if group_prefix else proposition_name
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        string_columns = ['id']
        for col in string_columns:
            df[col] = df[col].apply(lambda x: f"{prefix}{x}")
        dfs.append(df)
    
    # Concatenate all DataFrames in the list
    if dfs:
        concatenated_df = pd.concat(dfs, ignore_index=True)
        return concatenated_df
    else:
        print("No tables found or loaded.")

# Example usage
group_prefix = "group21_"  # !!!! Define the group prefix here !!!

df_item_id = load_and_concat_propositions(engine, group_prefix)
df_item_id

,id,question,scale
0,pre_2,"Overall, I am satisfied with how easy it is to...",7
1,pre_3,It is simple to use this system.,7
2,pre_4,I feel comfortable using this system.,7
3,pre_5,"The information (such as online help, on-scree...",7
4,pre_6,It is easy to find the infromation I need,7
5,pre_7,The information provded with the system is eas...,7
6,post_8,Provide useful suggestions,7
7,post_9,Make decisions easier,7
8,post_10,Are a good way to learn about different produc...,7
9,post_11,Offer suggestions that I might not have though...,7


In [5]:
df_item_id.to_csv(f'{group_prefix}item ids.csv', index=False)

In the next code cell we are selecting all the columns starting with "user"
except for the "users" column (as it does not include any relevant information
for us). Now we have a list of all the columns that we later want to retrieve
data for. By taking this approach, our code works for all kinds of different
columns starting with "user" since they are different from group to group.


In [6]:
# Select all the user columns
usercols = [n for n in tables if 'user' in n]

In [7]:
usercols

['group21_userallergies',
 'group21_userconsent',
 'group21_userdemograph',
 'group21_usermancheck',
 'group21_userpresatisfaction',
 'group21_users',
 'group21_usersatisfaction',
 'group21_usersatisfaction1',
 'group21_usersatisfaction2',
 'group21_usersatisfaction3',
 'group21_usersatisfaction4',
 'group21_userselection']

In the next cell we are defining a function that makes it possible for us later
to also add a prefix to column names which we will use later.


In [8]:
# Function to add prefix to all columns except 'userId'
def add_prefix_except_userid(df, prefix):
    return df.rename(columns={col: f"{prefix}{col}" if col != "userId" else col for col in df.columns})

In the following cell we are now finally retrieving our data: We take the names
of all the columns we identified as relevant (usercols), retrieve their content,
and put them all in a list. There are two types of tables that first need some
more work: userpresatisfaction(1) and usersatisfaction(1). These are the likert
scale items and they are stored in so-called long format (meaning that every
person appears in as many rows as are items in the likert scale, so if there is
a likert scale with 14 items there are 14 rows per study participant). As the
long format is not very useful for us to further work with the data, we convert
it into wide format (you can see the command below, .pivot) so that all the
tables are in the same format.


In [9]:
dfs = []
for col in usercols:
    query = f"SELECT * FROM {col}"
    tablecontent = pd.read_sql(query, engine)
    if 'userpresatisfaction' in col:
        tablecontent = tablecontent.pivot(index='userId', columns='questionId', values='value').reset_index()
        tablecontent.columns.name = None
        tablecontent = add_prefix_except_userid(tablecontent, 'pre_')
    elif 'usersatisfaction' in col:
        tablecontent = tablecontent.pivot(index='userId', columns='questionId', values='value').reset_index()
        tablecontent.columns.name = None
        tablecontent = add_prefix_except_userid(tablecontent, 'post_')
    elif 'userdemograph' in col: # !!! add more elif conditions and adapt table names if there are multiple extra 'time' columns !!!
        # drop column 'time'
        try:
            tablecontent = tablecontent.drop(columns=['time'])
        except:
            pass
    elif col == group_prefix + 'users':
        tablecontent = tablecontent.rename(columns={'time': 'start_time'})
    dfs.append(tablecontent)

In [10]:
dfs

[     userId  genre
 0         5      2
 1         5      2
 2       534      2
 3       535      2
 4       536      1
 ..      ...    ...
 218     720      4
 219     721      4
 220     722      2
 221     723      1
 222     724      4
 
 [223 rows x 2 columns],
     userId  consent
 0      532        1
 1      556        1
 2      558        1
 3      559        1
 4      583        1
 ..     ...      ...
 82     716        1
 83     715        1
 84     721        1
 85     722        1
 86     723        1
 
 [87 rows x 2 columns],
     userId  age  gender  device  education
 0      587   23       2       2          1
 1      588   23       2       2          2
 2      589   23       1       2          2
 3      590   20       1       2          2
 4      594   19       1       2          2
 ..     ...  ...     ...     ...        ...
 76     716   59       1       2          2
 77     715   54       2       2          4
 78     721   19       2       2          2
 79     722   2

Now we merge all the tables we collected in the list together on userId so that
we have one row per user with all the variables in one column each. We also
further remove some variables that are not of interest to us (such as time
variables) to further clean the dataframe


In [11]:
# Merge all DataFrames on the userId column
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on='userId', how="outer")

In [12]:
# check if there are duplicated 'userId' in merged_df and drop duplicates
merged_df.drop_duplicates(subset='userId', keep='last', inplace=True)

In [13]:
# Convert the start_time column to datetime format
merged_df['start_time'] = pd.to_datetime(merged_df['start_time'])

# remove start_time before 20 June 2024
merged_df = merged_df[merged_df['start_time'] >= '2024-06-20']

In [14]:
merged_df

,userId,genre,consent,age,gender,device,education,mancheck1,mancheck2,pre_1,...,post_36_y,post_37_y,post_38_y,post_39_y,post_40_y,post_41_y,post_42_y,conditie_y,calibrationId,recommendationId
579,560,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
580,561,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581,562,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
582,563,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
583,564,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,720,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,721,4.0,1.0,19.0,2.0,2.0,2.0,0.0,5.0,NaN,...,5.0,4.0,5.0,3.0,4.0,3.0,4.0,1.0,1914.0,414.0
767,722,2.0,1.0,22.0,2.0,2.0,4.0,0.0,2.0,NaN,...,5.0,5.0,6.0,5.0,3.0,6.0,4.0,0.0,929.0,2262.0
768,723,1.0,1.0,21.0,1.0,2.0,2.0,1.0,6.0,NaN,...,2.0,6.0,6.0,6.0,4.0,7.0,4.0,0.0,151.0,132.0


As a last step, we save the dataframe to a CSV file. We have now called this
dataframe final_df.csv, you can change the name if you want -- just make sure
you can find the dataframe after you have saved it. With this code, it is stored
in the same folder as your Python script.


In [15]:
merged_df.to_csv(f'{group_prefix}final_df.csv', index=False)